# Twitter Bot: Web Scraping a website for active stocks, how much it changed 1 to 6 stocks


## Importing all the required libraries

In [1]:
import tweepy
import requests  
import pandas as pd
import schedule
import time
from bs4 import BeautifulSoup 
import random
import tweepy
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import requests
from datetime import date





### References:
Tweepy Tutorial: https://realpython.com/twitter-bot-python-tweepy/
YouTube Tutorial: How to Schedule Tasks with Python https://www.youtube.com/watch?v=zF_DroDICaM
YouTube Tutorial: PYTHON VIRTUAL ENVIRONMENTS (Beginner's Guide to Python Lesson 9) https://www.youtube.com/watch?v=4jt9JPoIDpY
YouTube Tutorial: Web scraping in Python (Part 3): Building a dataset https://www.youtube.com/watch?v=v5DDW5dyfyc
YouTube:30 Days of Python - Day 21 - Twitter API with Tweepy - Python TUTORIAL (especially useful for 2nd Twitter account authentication) https://www.youtube.com/watch?v=dvAurfBB6Jk&list=LL&index=28&t=3s
YouTube:How to Web Scrape Any Table Online Using Python and BeautifulSoup https://www.youtube.com/watch?v=PY2I4UIZk48&list=LL&index=18&t=863s


### Entering in the Twitter creditentials so that authetication can be completed. Also creating the API object so that different methods can be called when necessary

In [3]:
   # Authenticate to Twitter
auth = tweepy.OAuthHandler("Zt00PBtXB16LM2gxRWoVBXoaP","UwN96llwc7D44yo8zEIPqKI9JOcxrVWaSHWExQRmVJNOAjRoe2")
auth.set_access_token("1358336778479022081-KBu5DLwt07lrfxFGOJe398rQkuFQZk","Rwx1tzqQwp41If2cCBjoM3hqNnN2w4Z1UcZT3V7rATYBN")
    # Create API object
api = tweepy.API(auth)
  #  api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

### Following cell includes code that
- scrapes a Yahoo Finance webpage and table that contains lists of stocks that are active is copied. A dataframe called df is created.
- The symbol column is placed in to a list
- '$' is added to the stock list so that they can become clickable and searchable on Twitter. For example AAPL will become $AAPL. 
- All the elements in the list are converted to a string. This string is called 'new_list'
- Another string called 'ding' is created which containts links to all the blogs. This uses the random function (random.click) to pick a random link.
- The Tweet is generated and published with the two strings put together. 
- The schedule package is used to call the two functions are run them. The job function is run every two hours and coding function is run every 3600 seconds. 
- A while loop is used to keep the schedule function running.


In [4]:
# The coding function is just an accessory to the main 'job' function. The purpose of the coding function is to see if the code is working ok.

def coding():
    print ("working")
    
# The job function scrapes the yahoo finance site for web scraping, creates two strings with important content and then publishes the tweet

   

    # Get the content on the yahoo finance webpage
r = requests.get('https://finance.yahoo.com/most-active')
   # print(r.text[0:20])
soup = BeautifulSoup(r.text, 'html.parser')
    # Fetch the table that lists the stocks
results = soup.find('section', attrs={'id':'screener-results'})
    #print(len(results))
    # Use the loop to generate the framework of the table in the yahoo finance. Focus here is on the headers
headers = []
for i in results.find_all('th'):
    title = i.text.strip()
    headers.append(title)
    # Create a data frame in the notebook    
df = pd.DataFrame(columns = headers)
   
    # This loops grabs the data inside the rows and puts it in the dataframe df
for j in results.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [tr.text.strip() for tr in row_data]
    length = len(df)
    df.loc[length] = row
    #display(df.head(5))
    # Select the symbol column and insert the elements in to a list
list_of_names = df['Symbol'][1:6].to_list()
    
    # Add/Concatenate a space and the dollar sign to the elements in the list 
    # new_list = [' $'+x for x in list_of_names]
    
    # The string randomly selects a link
  
    # Set = set([' https://y3khan.wordpress.com/2021/02/22/mistakes-new-traders-should-avoid-in-the-stock-market/', ' https://y3khan.wordpress.com/2021/02/16/what-stocks-should-i-buy/', ' https://y3khan.wordpress.com/2021/02/08/why-taking-out-a-loan-to-purchase-stocks-is-not-a-dangerous-idea/',' https://y3khan.wordpress.com/2021/02/01/markets-are-at-all-time-highs-should-i-sell-now/', ' https://y3khan.wordpress.com/2021/01/25/three-bubbles-that-burst-in-the-last-decade/', ' https://y3khan.wordpress.com/2021/01/19/three-core-issues-asset-teams-mismanage-in-the-oil-and-gas-industry/', ' https://y3khan.wordpress.com/2021/01/14/quick-look-at-my-transaction-costs/', ' https://y3khan.wordpress.com/2020/05/15/selling-when-markets-are-in-turmoil/' ])
    # ding= random.choices(list(Set),k= 1)
    # ding2 = ' '.join([str(elem) for elem in ding])
today = date.today()
a = date(date.today().year, 1, 1)
    # dd/mm/YY
d1 = today.strftime("%Y-%m-%d")
d2 = a.strftime("%Y-%m-%d")
data = yf.download(list_of_names,start=d2, end=d1)
data1=data.dropna()
Prices= data1['Adj Close']
Prices1 = ((Prices - Prices.iloc[0])/Prices.iloc[0]) * 100
lastr= Prices1.round(1).tail(1)
header= list(lastr)
lastlist= lastr.values.tolist()
master=[]
for sym, num in zip(header, lastlist[0]):
    if num >0 :
        master.append((''.join(("$", sym, " is up ",  str(num),  "%" , ' \n'))))  
    else:
           master.append((''.join(("$", sym, " is down ",  str(num),  "%" , ' \n'))))  
  

# This code converts each element in the list to a string
listToStr1 = ''.join([str(elem) for elem in master])
    #divide all dataframe by first line of data to enable comparison

    # This code converts each element in the list to a string
                      
    # This code converts each element in the list to a string
    #listToStr = ' '.join([str(elem) for elem in new_list])
    #print(listToStr)
    
    # This code combines the text, the two strings and publishes the tweet
api.update_status("This year: \n" + listToStr1)



[*********************100%***********************]  5 of 5 completed


Status(_api=<tweepy.api.API object at 0x7fa8f3350df0>, _json={'created_at': 'Fri Jan 07 04:30:54 +0000 2022', 'id': 1479309547378462722, 'id_str': '1479309547378462722', 'text': 'This year: \n$AAPL is down -3.9% \n$AMD is down -9.4% \n$BAC is up 2.2% \n$LCID is down -10.4% \n$T is up 3.1%', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [{'text': 'AAPL', 'indices': [12, 17]}, {'text': 'AMD', 'indices': [33, 37]}, {'text': 'BAC', 'indices': [53, 57]}, {'text': 'LCID', 'indices': [70, 75]}, {'text': 'T', 'indices': [92, 94]}], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://help.twitter.com/en/using-twitter/how-to-tweet#source-labels" rel="nofollow">yk303</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1358336778479022081, 'id_str': '1358336778479022081', 'name': '🚀🚀 Trending Stocks 🚀🚀', 'screen_name': 'get_blow', 'location': 'Ne